In [1]:
import os
import sys
parent = os.path.realpath(os.path.join(os.path.abspath(''), os.pardir))
sys.path.append(parent)
import numpy as np
from databases.sql_connect import create_connection
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option("mode.chained_assignment", None) 
sns.set_theme()
sns.set(context="paper", style="darkgrid", font="Times New Roman",font_scale = 2.2)
cm = 1/2.54
DB_CORR = os.path.join(parent,"databases", "TEMPRO_DB230515_Corr.db")
assert(os.path.exists(DB_CORR))
FONTSIZE = 8

In [8]:

def build_material_contribution_charts(
    GROUP = 5000,
    impact= "CML-climate change (GWP 100a)",
    y_full_label = "CO2"):


    DB_DIR_TEMP = DB_CORR
    conn = create_connection(DB_DIR_TEMP)

    query = f'SELECT * FROM [{GROUP}LCAResults]'
    df_lca = pd.read_sql_query(query, conn)

    query7000= "SELECT * FROM [7000Recovery]"
    df7000 = pd.read_sql_query(query7000, conn)

    queryEDIP = "SELECT * FROM [C:Resource Indicators]"
    dfEDIP = pd.read_sql_query(queryEDIP, conn)

    query1000 = "SELECT * FROM [1000Materials]"
    df1000 = pd.read_sql_query(query1000, conn)

    listMaterials = []
    for mat in df1000["Name"]:
            listMaterials.append(mat)

    list_7000 = []
    for idx in df7000.index:
        if not ":" in df7000.loc[idx,"Name"]:
            name= str(df7000.loc[idx,"ID"])+ ":"+ df7000.loc[idx,"Name"]
            list_7000.append(name)

    #The first thing to do is to make this table horizontal
    ids=df_lca["ProductSystemID"].unique()
    impacts = df_lca["Category"].unique()

    new_df=pd.DataFrame()
    new_df["Impacts"]=impacts
    new_df.sort_values(by="Impacts", ascending=True)
    new_df["Keep"]=True

    for idx in new_df.index:
        if new_df.loc[idx,"Impacts"][5:] not in listMaterials:
            new_df.loc[idx,"Keep"]=False
        if new_df.loc[idx,"Impacts"][:4] =="EDIP":
            new_df.loc[idx,"Keep"]=False
                
    for id in ids:
        df_id=pd.DataFrame()
        df_id=df_lca[df_lca["ProductSystemID"]==id]
        df_id.sort_values(by="Category", ascending=True)
        new_df[id]=list(df_id["Result"])

    new_df["Impacts"] = [x[5:] for x in new_df["Impacts"]]
    new_df.set_index("Impacts", inplace=True)
    new_df=new_df[new_df["Keep"]==True]
    new_df.drop(columns=["Keep"], inplace=True)
    query = 'SELECT * FROM [1000LCAResults]'
    df_material_impacts= pd.read_sql_query(query, conn)
    df_material_impacts=df_material_impacts[df_material_impacts["Category"]==impact]


    df_material_impacts["in Recovery"]=False
    for idx in df_material_impacts.index:
        if df_material_impacts.at[idx,"ProductSystemName"] in listMaterials:
            df_material_impacts.at[idx,"in Recovery"] = True

    df_material_impacts=df_material_impacts[df_material_impacts["in Recovery"]==True]
    df_material_impacts.set_index("ProductSystemName", inplace=True)
    df_material_impacts=df_material_impacts[["Category","Result"]]

    #First, get the impact from the material flow LCA results
    df_impact= df_lca[df_lca["Category"]==impact]
    df_impact_f=df_impact[["ProductSystemID",	"ProductSystemName"	,"Category","Result"]]
    df_impact_f["Subsubcat"]=np.nan

    #df_impact_f=df_impact_f[df_impact_f["ProductSystemID"]<2700]
    df_subsubcats = pd.read_excel("2000subsubcats.xlsx", engine="openpyxl", sheet_name="Subsubcat")
    df_subsubcats.set_index("ID", inplace=True)    

    df_impact_f.set_index("ProductSystemID",inplace=True)

    for idx in df_impact_f.index:
        if idx in df_subsubcats.index:
            df_impact_f.at[idx,"Subsubcat"]= df_subsubcats.at[idx,"Subsubcat New"]

    df_impact_f=df_impact_f[df_impact_f["Subsubcat"].notna()]
    df_impact_f["MaterialsContribution"]=np.nan
    df_impact_f_small = deepcopy(df_impact_f)

    for ps in df_impact_f_small.index:
        df_ps_material_use=new_df[[ps]]
        df_ps_material_use["Material_Impact"]=0
        df_ps_material_use["Material_Contribution"]=0

        for idx in df_ps_material_use.index:
            if idx in df_material_impacts.index:
                flow_val=df_material_impacts.at[idx,"Result"]
                df_ps_material_use.at[idx,"Material_Impact"]=flow_val

        df_ps_material_use["Material_Contribution"]=df_ps_material_use[ps]*df_ps_material_use["Material_Impact"]
        total_impact_contribution=df_ps_material_use["Material_Contribution"].sum()
        df_impact_f_small.at[ps,"MaterialsContribution"]= total_impact_contribution

    df_impact_f_small["OthersContribution"]=df_impact_f_small["Result"]-df_impact_f_small["MaterialsContribution"]
    df_impact_f_small=df_impact_f_small[df_impact_f_small["MaterialsContribution"]>0]
    #df_impact_f_small=df_impact_f_small[df_impact_f_small["OthersContribution"]>0]
    df_impact_f_small["Mat_Perc"]=df_impact_f_small["MaterialsContribution"]/df_impact_f_small["Result"]
    df_impact_f_small.sort_values(by="Mat_Perc", ascending=False, inplace=True)
    #df_impact_f_small.to_csv(f"MaterialContributions\materials_contributions_{GROUP}-{impact}.csv")

    if GROUP ==6000:
        # use the whole data center as a reference
        df_subsubcats["Ref Unit"] = [1 for x in df_subsubcats["Ref Unit"]]
    for idx in df_impact_f_small.index:
        if idx in df_subsubcats.index:
            if df_subsubcats.at[idx,"Ref Unit"]>0:
                df_impact_f_small.at[idx,"Result"]=df_impact_f_small.at[idx,"Result"]/df_subsubcats.at[idx,"Ref Unit"]
                df_impact_f_small.at[idx,"MaterialsContribution"]=df_impact_f_small.at[idx,"MaterialsContribution"]/df_subsubcats.at[idx,"Ref Unit"]
                df_impact_f_small.at[idx,"OthersContribution"]=df_impact_f_small.at[idx,"OthersContribution"]/df_subsubcats.at[idx,"Ref Unit"]


    df_res = deepcopy(df_impact_f_small)
    df_res.index = df_res.index.map(str)
    #df_res.sort_values(by=["Subsubcat", "Result"], ascending=True, inplace=True)
    df_res.sort_values(by=["Result"], ascending=True, inplace=True)

    if GROUP in [3000,5000]:
        df_res=df_res[df_res["Result"]!=min(df_res["Result"])]
    df_res["Order"] =range(1,df_res.shape[0]+1)

    fig,ax= plt.subplots(figsize=(22,6.5))
    p=sns.barplot(x="Order", y="Result", hue="Subsubcat", data=df_res, ax=ax, alpha=0.3,dodge=False)
    p.legend_.remove()
    ax.set_yscale('log')
    p2=sns.barplot(x="Order", y="MaterialsContribution", hue="Subsubcat", data=df_res, ax=ax,alpha=1, dodge=False)
    handles, labels = ax.get_legend_handles_labels()
    ax.get_legend().remove()
    titles_dict={
        2000:"Pieces",
        3000:"Parts",
        4000:"Devices",
        5000:"Systems",
        6000:"Data Centers" }
    df_res.to_csv(f"MaterialContributions\materials_contributions_{GROUP}-{impact}.csv")
    ax.legend(handles[int(len(handles)/2):], labels[int(len(handles)/2):],bbox_to_anchor=(1, 1))
    #ax.legend(ncol=3,           loc='lower center', borderaxespad=0.)
    #ax.set_title(titles_dict[GROUP], fontweight='bold', fontsize=18)
    plt.xticks([])
    ax.set_xlabel("")
    group =titles_dict[GROUP]
    y_full_label = f"{group}\n{y_full_label}"
    ax.set_ylabel(y_full_label)
    fig.savefig(f"MaterialContributions\materials_contributions_{GROUP}-{impact}.png", bbox_inches='tight')
    plt.close()

build_material_contribution_charts(
                GROUP = 6000,
                impact= "CML-climate change (GWP 100a)",
                y_full_label = "test")
    

In [9]:
df_impact_names=pd.read_csv("impacts_names.csv")
df_impact_names=df_impact_names[df_impact_names["Use"].notna()]

for idx in df_impact_names.index:
    impact = df_impact_names.at[idx,"Name"]
    print(impact)
    y_label = df_impact_names.at[idx,"LongName"]
    unit = df_impact_names.at[idx,"Unit"]
    y_label_full = f"{y_label} [{unit}]"
    if any (x in impact for x in ["CED", "EI-Minerals", "CExD - minerals","GPR-Minerals- Total","ReCiPe Endpoint (E - A) - resources (total)"]):
        for g in [6000]:# [3000,4000,5000,6000]:
        #try:
            build_material_contribution_charts(
                GROUP = g,
                impact= impact,
                y_full_label = y_label_full)
        # except Exception as e:
        #     print(e)
        #     print("Failed: ",  g, impact)





ADP-Minerals- Total
CED - total
CExD - minerals
CML-climate change (GWP 100a)
CML-human toxicity (HTP 100a)
EcoScarcity2013 - total
EI-Minerals- Total
GPR-Minerals- Total
ReCiPe Endpoint (E - A) - resources (total)
ReCiPe Endpoint (E - A) - total (total)
SR-Minerals- Total
